# Get Quarterly Revenue Growth for Microsoft

This notebook fetches quarterly revenue data for Microsoft (MSFT) from the Rice Data Portal and calculates year-over-year growth rates.

In [ ]:
from rice_data_client import RiceDataClient
import pandas as pd

# Initialize client with your access token
ACCESS_TOKEN = "your_access_token_here"  # Replace with your token from data-portal.rice-business.org
client = RiceDataClient(access_token=ACCESS_TOKEN)

## Query Quarterly Revenue Data

We'll use the SF1 table with dimension MRQ (Most Recent Quarterly) to get quarterly revenue data.
The `reportperiod` column contains the fiscal period end date.

In [ ]:
# Query quarterly revenue for last 5 years
sql = """
SELECT 
    ticker,
    reportperiod::DATE as reportperiod,
    fiscalperiod,
    revenue
FROM sf1
WHERE ticker = 'MSFT'
  AND dimension = 'MRQ'
  AND reportperiod::DATE >= CURRENT_DATE - INTERVAL '5 years'
ORDER BY reportperiod
"""

df = client.query(sql)

# Convert reportperiod to datetime
df['reportperiod'] = pd.to_datetime(df['reportperiod'])

print(f"Retrieved {len(df)} quarterly observations")
print(f"Date range: {df['reportperiod'].min()} to {df['reportperiod'].max()}")

In [ ]:
# Preview the data
df.head(10)

## Calculate Year-over-Year Revenue Growth

For quarterly data, year-over-year growth compares to the same quarter in the prior year (lag of 4 quarters).

In [ ]:
# Calculate YoY growth rate
df = df.sort_values('reportperiod').reset_index(drop=True)

# Lag revenue by 4 quarters for YoY comparison
df['revenue_lag4'] = df['revenue'].shift(4)

# Calculate YoY growth percentage
df['yoy_growth_pct'] = ((df['revenue'] - df['revenue_lag4']) / df['revenue_lag4']) * 100

# Round for readability
df['yoy_growth_pct'] = df['yoy_growth_pct'].round(2)

print(f"\nQuarterly Revenue Growth for Microsoft:")
print(df[['reportperiod', 'fiscalperiod', 'revenue', 'yoy_growth_pct']].tail(8))

## Save Data as Parquet File

In [ ]:
# What filename would you like to use to save this data?
filename = "msft_quarterly_revenue_growth.parquet"

df.to_parquet(filename)
print(f"Data saved to {filename} ({len(df)} rows)")

## Visualize Revenue Growth

In [ ]:
import matplotlib.pyplot as plt

# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Plot 1: Quarterly Revenue
ax1.plot(df['reportperiod'], df['revenue'] / 1000, linewidth=2, color='#0078D4')
ax1.set_title('Microsoft Quarterly Revenue', fontsize=14, fontweight='bold')
ax1.set_xlabel('Report Period', fontsize=12)
ax1.set_ylabel('Revenue ($B)', fontsize=12)
ax1.grid(True, alpha=0.3)

# Plot 2: YoY Growth Rate
# Drop NaN values for plotting
df_growth = df.dropna(subset=['yoy_growth_pct'])
colors = ['green' if x >= 0 else 'red' for x in df_growth['yoy_growth_pct']]
ax2.bar(df_growth['reportperiod'], df_growth['yoy_growth_pct'], color=colors, alpha=0.7)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.set_title('Microsoft YoY Revenue Growth Rate', fontsize=14, fontweight='bold')
ax2.set_xlabel('Report Period', fontsize=12)
ax2.set_ylabel('YoY Growth (%)', fontsize=12)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Summary statistics
print("\nRevenue Growth Summary Statistics:")
print(f"Mean YoY Growth: {df_growth['yoy_growth_pct'].mean():.2f}%")
print(f"Median YoY Growth: {df_growth['yoy_growth_pct'].median():.2f}%")
print(f"Min YoY Growth: {df_growth['yoy_growth_pct'].min():.2f}%")
print(f"Max YoY Growth: {df_growth['yoy_growth_pct'].max():.2f}%")
print(f"Std Dev: {df_growth['yoy_growth_pct'].std():.2f}%")